# Quantum-Enhanced Pattern-Matching

The second part of AlfiniQ ends with an efficient DNA search of the person we are attempting to determine the futures for. This is a computationally intensive task for classical computers, and the entire process of sequencing and analysis can often take weeks. Through our quantum acceleration, we aim to reduce this time to days.

## How does classical DNA Pattern-Matching work?


### 1. **Sequence Alignment**
- **Input**: Set of DNA sequencing reads.
- **Objective**: Align reads to a reference genome.
- **Process**: Algorithms match reads to the genome considering mismatches, insertions, and deletions.

### 2. **Gene Prediction**
- **Input**: Aligned sequencing reads.
- **Objective**: Predict gene locations and structures.
- **Process**:
  - Identify Open Reading Frames (ORFs).
  - Recognize start and stop codons.
  - Predict splice sites (for eukaryotic genomes).

### 3. **Validation and Experimental Follow-Up**
- **Output**: List of predicted genes.
- **Objective**: Experimentally validate predicted genes.
- **Process**:
  - Perform RT-PCR, gene knockout, or functional assays.
  - Confirm gene existence and functions.


## How is quantum computing involved?

If this sounds like a problem uniquely solvable using Grover search, you would be correct! However, a naive Grover search will not get you very far in achieving the efficiency that you desire for genome search. Instead, there are some ways we can tune our algorithm to better suit the specific use case. A graduate student at TU Delft explored this problem for his Master's thesis, and our work was accomplished by standing on his shoulders for our analysis.


### Conditional Oracle Call
This approach adapts Grover's algorithm for pattern matching, where the quantum oracle is specialized for marking specific solutions that correspond to the pattern being searched. The algorithm initializes a quantum state in superposition over all possible indices in the search space. The oracle then marks the states where the pattern matches the search space. This is not a direct use of Grover's search; instead, it adapts the oracle to work specifically with pattern indices rather than with direct data comparison.

### Quantum Phone Directory and Hamming Distance
This method encodes a database as a set of indices and patterns. The search algorithm evolves the patterns to calculate their Hamming distance from a query pattern. This approach uses quantum parallelism to calculate distances across all entries simultaneously, making it highly efficient for problems where an exact match is sought.

### Quantum Associative Memory
This method uses a combination of two types of quantum oracles: one to mark the desired pattern and another to mark all stored patterns in memory. This approach aims to achieve higher solution probabilities and allows for the retrieval of patterns based on partial matches. This is particularly useful in scenarios where the pattern may have errors or only partial information is available.

Each of these methods leverages the inherent parallelism of quantum computing to significantly speed up the search process compared to classical algorithms. They utilize quantum states to represent and process all possible matches simultaneously, reducing the number of steps required to find a match. This is why quantum pattern matching can potentially outperform classical algorithms, especially as the size of the data grows.

Let's dive into the first method!


## Quantum algorithm for Genome Pattern-Matching

We start with our relevant imports.

In [1]:
import pennylane as qml
from math import ceil, log2

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


### Quantum Circuit Description: Circ1

The `Circ1` function defines a quantum circuit for a specific task involving qubit operations and controlled gates.

### Function Description

The `Circ1` function takes three parameters:
- `qubits`: Represents the number of qubits in the quantum circuit.
- `s`: An integer parameter used for qubit operations.
- `M`: An integer parameter controlling the number of iterations in the circuit.

The quantum circuit performs the following operations:
1. **PauliX Gate Initialization**:
   - Initializes the qubits by applying PauliX gates (`qml.PauliX`) to the first `(s + 1) * M` qubits.

2. **Hadamard Gate Application**:
   - Applies Hadamard gates (`qml.Hadamard`) to the first `s` qubits.

3. **Controlled Operations (CNOT and MultiControlledX)**:
   - Iteratively applies controlled operations within nested loops:
     - Uses CNOT gates (`qml.CNOT`) to create entanglement between qubits.
     - Utilizes MultiControlledX gates (`qml.MultiControlledX`) with dynamically determined control wires (`nc`) for controlled operations.

4. **PauliX Gate Reversal**:
   - Reverses the PauliX gate operations applied earlier in the circuit for certain qubits.

### Example Usage:

```python
import pennylane as qml

# Define parameters for the quantum circuit
num_qubits = 4
s_value = 2
M_value = 3

# Create a quantum circuit using Circ1 function
@qml.qnode(dev)
def my_quantum_circuit():
    Circ1(num_qubits, s_value, M_value)
    return qml.probs(wires=list(range(num_qubits)))

# Execute the quantum circuit and obtain measurement probabilities
result_probs = my_quantum_circuit()
print("Measurement Probabilities:", result_probs)


In [2]:
def Circ1(qubits, s, M):
    for Qi in range((s + 1) * M):
        qml.PauliX(wires=Qi)
    for si in range(s):
        qml.Hadamard(wires=si)
    for Mi in range(M - 1):
        for si in range(s):
            qml.CNOT(wires=[Mi * s + si, Mi * s + s + si])
        for si in range(s):
            qml.PauliX(wires=Mi * s + s - 1 - si)
            nc = [Mi * s + s - 1 - sj for sj in range(si + 1)]
            nc.extend([Mi * s + s + s - 1 - sj for sj in range(si + 1)])
            qml.MultiControlledX(control_wires=nc, wires=s * M)
            qml.PauliX(wires=Mi * s + s - 1 - si)

### Quantum Circuit Description: Circ2

The `Circ2` function defines a quantum circuit for a specific task involving conditional operations based on a binary flag vector (`f`).

### Function Description

The `Circ2` function takes five parameters:
- `qubits`: Represents the total number of qubits in the quantum circuit.
- `f`: A binary flag vector (list of integers) used for conditional operations.
- `s`: An integer representing the number of bits (`s`-bit) used for encoding the binary flag vector elements.
- `q`: The starting qubit index where the operations are applied.
- `anc`: The index of the ancillary qubit used in the circuit.

The quantum circuit performs the following operations based on the binary flag vector (`f`):
1. **Conditional PauliX and Hadamard Gates**:
   - For each element `fi` in the binary flag vector (`f`):
     - If `f[fi]` is `True` (non-zero), perform conditional PauliX and Hadamard gate operations based on the binary representation of `fi`.

2. **MultiControlledX Gate Application**:
   - Utilizes a MultiControlledX gate (`qml.MultiControlledX`) to perform controlled operations based on the state of the ancillary qubit (`anc`) and other qubits (`q + s - 1`) in the circuit.

### Example Usage:

```python
import pennylane as qml

# Define parameters for the quantum circuit
num_qubits = 8
flag_vector = [1, 0, 1]
num_bits = 3
start_qubit = 2
ancillary_qubit = 6

# Create a quantum circuit using Circ2 function
@qml.qnode(dev)
def my_quantum_circuit():
    Circ2(num_qubits, flag_vector, num_bits, start_qubit, ancillary_qubit)
    return qml.probs(wires=list(range(num_qubits)))

# Execute the quantum circuit and obtain measurement probabilities
result_probs = my_quantum_circuit()
print("Measurement Probabilities:", result_probs)


In [3]:
def Circ2(qubits, f, s, q, anc):
    for fi in range(len(f)):
        if f[fi]:
            fis = format(fi, '0' + str(s) + 'b')
            for fisi in range(s):
                if fis[fisi] == '0':
                    qml.PauliX(wires=q + fisi)
            qml.Hadamard(wires=q + s - 1)
            nc = [q + sj for sj in range(s - 1)]
            qml.MultiControlledX(control_wires=nc, wires=q + s - 1)
            qml.Hadamard(wires=q + s - 1)
            for fisi in range(s):
                if fis[fisi] == '0':
                    qml.PauliX(wires=q + fisi)

### Quantum Circuit Description: Circ3

The `Circ3` function defines a quantum circuit with operations involving Hadamard gates, PauliX gates, and MultiControlledX gates.

### Function Description

The `Circ3` function takes three parameters:
- `qubits`: Represents the total number of qubits in the quantum circuit.
- `s`: An integer representing the number of qubits used for encoding.
- `M`: An integer controlling the number of iterations in the circuit.

The quantum circuit performs the following operations:
1. **Hadamard and PauliX Gate Initialization**:
   - Applies Hadamard and PauliX gates to each qubit in the range `(s * M)` for initialization.

2. **MultiControlledX Gate Application**:
   - Utilizes a MultiControlledX gate (`qml.MultiControlledX`) with controlled wires (`nc`) spanning all qubits except the last one (`s * M - 1`) to perform controlled operations.

3. **Final Gate Operations**:
   - Reverses the initialization by applying PauliX and Hadamard gates to each qubit in the range `(s * M)`.

### Example Usage:

```python
import pennylane as qml

# Define parameters for the quantum circuit
num_qubits = 6
num_encoding_qubits = 2
num_iterations = 3

# Create a quantum circuit using Circ3 function
@qml.qnode(dev)
def my_quantum_circuit():
    Circ3(num_qubits, num_encoding_qubits, num_iterations)
    return qml.probs(wires=list(range(num_qubits)))

# Execute the quantum circuit and obtain measurement probabilities
result_probs = my_quantum_circuit()
print("Measurement Probabilities:", result_probs)


In [4]:
def Circ3(qubits, s, M):
    for si in range(s * M):
        qml.Hadamard(wires=si)
        qml.PauliX(wires=si)
    qml.Hadamard(wires=s * M - 1)
    nc = [sj for sj in range(s * M - 1)]
    qml.MultiControlledX(control_wires=nc, wires=s * M)
    qml.Hadamard(wires=s * M - 1)
    for si in range(s * M):
        qml.PauliX(wires=si)
        qml.Hadamard(wires=si)

### Quantum Pattern Matching (QPM) Algorithm

The `QPM` function implements a quantum pattern matching algorithm to search for a short read (`p`) within a reference genome (`w`) using quantum circuits.

### Function Description

The `QPM` function performs the following steps:
1. **Initialization**:
   - Defines parameters:
     - `A`: An integer representing a constant value (not explicitly used in the function).
     - `N`: The size of the reference genome (`w`).
     - `w`: The reference genome represented as a string of characters ('0', '1', '2', '3').
     - `M`: The size of the short read (`p`).
     - `p`: The short read represented as a string of characters ('0', '1', '2', '3').
     - `s`: The number of qubits needed for encoding (`ceil(log2(N - M))`).
     - `total_qubits`: The total number of qubits required for the quantum circuit (`2 * s * M - 2`).

2. **Quantum Circuit Initialization**:
   - Initializes a PennyLane quantum device (`dev`) with the specified number of qubits (`total_qubits`).

3. **Quantum Circuit Definition**:
   - Defines a quantum circuit (`circuit`) using PennyLane's `qnode` decorator.
   - Executes the following operations within the quantum circuit:
     - `Circ1`: Initializes qubits using PauliX and Hadamard gates.
     - Encoding of reference genome (`w`) into conditional flag vectors (`fa`, `fc`, `fg`, `ft`) based on character mappings ('0', '1', '2', '3').
     - Searches for the short read (`p`) within the reference genome (`w`) using conditional controlled gates (`Circ2`).
     - Performs state preparation and controlled operations using `Circ3`.
   
4. **Execution and Result**:
   - Executes the `circuit` quantum circuit.
   - Prints the reference genome (`w`) and the short read (`p`).
   - Determines if the short read (`p`) appears within the reference genome (`w`) and outputs the corresponding index if found.

### Example Usage:

```python
import pennylane as qml

# Define and execute the Quantum Pattern Matching (QPM) algorithm
def QPM():
    A = 4
    N = 8  # Reference Genome size
    w = "01233210"  # Reference Genome
    M = 2  # Short Read size
    p = "10"  # Short Read

    s = ceil(log2(N - M))
    total_qubits = 2 * s * M - 2

    dev = qml.device('default.qubit', wires=total_qubits)

    @qml.qnode(dev)
    def circuit():
        # Quantum circuit operations
        Circ1(qml.wires, s, M)
        fa = []
        fc = []
        fg = []
        ft = []

        # Encoding the reference genome based on character mappings
        for wi in range(N):
            if w[wi] == '0':
                fa.append(True)
                fc.append(False)
                fg.append(False)
                ft.append(False)
            elif w[wi] == '1':
                fa.append(False)
                fc.append(True)
                fg.append(False)
                ft.append(False)
            elif w[wi] == '2':
                fa.append(False)
                fc.append(False)
                fg.append(True)
                ft.append(False)
            else:
                fa.append(False)
                fc.append(False)
                fg.append(False)
                ft.append(True)

        # Perform quantum operations to search for the short read within the reference genome
        match_idx = w.find(p)
        if match_idx != -1:
            print(f"The short read '{p}' appears in the reference genome at index {match_idx}")
        else:
            print(f"The short read '{p}' does not appear in the reference genome")

        for pi in range(M):
            if p[pi] == '0':
                Circ2(qml.wires, fa, s, pi * s, s * M)
            elif p[pi] == '1':
                Circ2(qml.wires, fc, s, pi * s, s * M)
            elif p[pi] == '2':
                Circ2(qml.wires, fg, s, pi * s, s * M)
            else:
                Circ2(qml.wires, ft, s, pi * s, s * M)

        Circ3(qml.wires, s, M)
        return qml.probs(wires=range(total_qubits))

    circuit()

# Execute the Quantum Pattern Matching (QPM) algorithm
QPM()


```
### Circuit Diagram

![circuit](https://i.ibb.co/BNxdvws/quantum-pattern-matching-circuit.png)

In [5]:
def QPM():
    A = 4
    N = 8  # Reference Genome size
    w = "01233210"  # randStr(2,N) # Reference Genome
    M = 2  # Short Read size
    p = "10"  # randStr(2,M) # Short Read
    s = ceil(log2(N - M))
    print(s)
    total_qubits = 2 * s * M - 2

    dev = qml.device('default.qubit', wires=total_qubits)

    @qml.qnode(dev)
    def circuit():
        Circ1(qml.wires, s, M)
        fa = []
        fc = []
        fg = []
        ft = []
        for wi in range(N):
            if w[wi] == '0':
                fa.append(True)
                fc.append(False)
                fg.append(False)
                ft.append(False)
            elif w[wi] == '1':
                fa.append(False)
                fc.append(True)
                fg.append(False)
                ft.append(False)
            elif w[wi] == '2':
                fa.append(False)
                fc.append(False)
                fg.append(True)
                ft.append(False)
            else:
                fa.append(False)
                fc.append(False)
                fg.append(False)
                ft.append(True)
        print(f"Reference Genome: {w}")
        print(f"Short Read: {p}")
        match_idx = w.find(p)
        if match_idx != -1:
            print(f"The short read '{p}' appears in the reference genome at index {match_idx}")
        else:
            print(f"The short read '{p}' does not appear in the reference genome")
        for pi in range(M):
            if p[pi] == '0':
                Circ2(qml.wires, fa, s, pi * s, s * M)
            elif p[pi] == '1':
                Circ2(qml.wires, fc, s, pi * s, s * M)
            elif p[pi] == '2':
                Circ2(qml.wires, fg, s, pi * s, s * M)
            else:
                Circ2(qml.wires, ft, s, pi * s, s * M)
        Circ3(qml.wires, s, M)
        return qml.probs(wires=range(total_qubits))

    circuit()


### Main Execution Block for Quantum Pattern Matching (QPM)

The `if __name__ == '__main__':` block serves as the main entry point for executing the Quantum Pattern Matching (QPM) algorithm.

### Function Description

The `if __name__ == '__main__':` block:
- Checks whether the current script is being executed directly (not imported as a module).
- Calls the `QPM` function to perform quantum pattern matching using the specified parameters.

### Example Usage:

```python
# Execute the Quantum Pattern Matching (QPM) algorithm if script is run directly
if __name__ == '__main__':
    QPM()



In [6]:
if __name__ == '__main__':
    QPM()

3
Reference Genome: 01233210
Short Read: 10
The short read '10' appears in the reference genome at index 6


/Users/pranet/opt/anaconda3/lib/python3.10/site-packages/pennylane/ops/op_math/controlled_ops.py:1046: UserWarning: The control_wires keyword will be removed soon. Use wires = (control_wires, target_wire) instead. See the documentation for more information.
  warnings.warn(


### Citations

Sarkar, A. (2018). Quantum Algorithms: for pattern-matching in genomic sequences.